# Build and train a RNN model for time series data

Author: Robin Baumann

## Import and Load Dependencies

In [ ]:
%%capture

# Load TensorFlow and shwo versions
!pip install tensorflow==2.5
import tensorflow as tf

# Load Tensorflow optimizations
!pip install tensorflow_model_optimization
import tensorflow_model_optimization as tfmot

# Load TensorFlow Lites C converter
!pip install
from tensorflow.lite.python.util import convert_bytes_to_c_source

# Numpy 
!pip install numpy
import numpy as np

# Matplotlib 
!pip install matplotlib
import matplotlib.pyplot as plt

# Python's math library
!pip install math
import math

# Define paths to model files
!pip install os
import os
!pip install shutil
import shutil

In [ ]:
# Print tensorflow versions
print('TF Version', tf.__version__)
print('Keras Version', tf.keras.__version__)

In [ ]:
# Clear backend and set random seeds for reproducability
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

## Configure directories

In [ ]:
# Folder for all Models
MODELS_DIR = 'models/'
if os.path.exists(MODELS_DIR):
    shutil.rmtree(MODELS_DIR)
os.mkdir(MODELS_DIR)

# Folder for all Data
DATA_DIR = 'data/'
if os.path.exists(DATA_DIR):
    shutil.rmtree(DATA_DIR)
os.mkdir(DATA_DIR)

# Tensorboard logs
LOG_DIR = 'logs/'
if os.path.exists(LOG_DIR):
    shutil.rmtree(LOG_DIR)
os.mkdir(LOG_DIR)
    
# Directorys for Models
MODEL_NAME = 'time_series_model_rnn'
MODEL_TF = MODELS_DIR + MODEL_NAME
MODEL_KERAS = MODEL_TF + '.h5'
MODEL_TF_QUANT = MODELS_DIR + MODEL_NAME + '_quant'
MODEL_KERAS_QUANT = MODEL_TF_QUANT + '.h5'
MODEL_TFLITE = MODELS_DIR + MODEL_NAME + '.tflite'
MODEL_TFLITE_HEADER = MODELS_DIR + MODEL_NAME + '.h'
MODEL_TFLITE_QUANT = MODELS_DIR + MODEL_NAME + '_quant.tflite'
MODEL_TFLITE_HEADER_QUANT = MODELS_DIR + MODEL_NAME + '_quant.h'

## Dataset

### 1. Generate Data

#### Sunspot Data

In [ ]:
# # import wget to load Data
# !pip install wget
# import wget

# # Get Test Dataset
# fielname = wget.download('https://storage.googleapis.com/laurencemoroney-blog.appspot.com/Sunspots.csv', out=DATA_DIR)

# # import Dependencie to handel csv files
# import csv

# # Read Data from zip
# time_step = []
# sunspots = []
# with open(fielname) as csvfile:
#   reader = csv.reader(csvfile, delimiter=',')
#   next(reader)
#   for row in reader:
#     sunspots.append(float(row[2]))
#     time_step.append(int(row[0]))

# # Define Data
# time = np.array(time_step)
# true_values = np.array(sunspots)

# # define Inputs
# input_1 = true_values.copy()
# for t in range(len(input_1)):
#   input_1[t] += 0.1 * input_1[t - 10]
#   input_1[t] += 0.8 * input_1[t - 2]
# input_1 = input_1*42

# input_2 = true_values.copy()
# input_2 +=  max(input_2)*np.cos(time*np.pi)
# input_2 = input_2+42.2

#### Sinus Example

In [ ]:
# Create time vector for a Period of 10 pi
time = np.linspace(0, 10*np.pi, num=1000)

# Create sinus for time interval (with noise)
true_values = np.sin(time)

# Create cos as inputs for network (with noise)
input_1 = np.sin(time)+2*np.cos(time)**2
#input_1 += 0.05 * np.random.randn(*input_1.shape)
input_2 = np.sin(time)**2*np.cos(time)
#input_2 += 0.05 * np.random.randn(*input_2.shape)

# Plot the data
# True data
plt.figure(figsize=(15,3))
plt.title('True Values (logits)')
plt.plot(time, true_values, 'k-', label="true_values")
plt.legend()
plt.show()

# input_1
plt.figure(figsize=(15,3))
plt.title('Input 1')
plt.plot(time, input_1, 'k-', label="input_1")
plt.legend()
plt.show()
# input_2
plt.figure(figsize=(15,3))
plt.title('Input 2')
plt.plot(time, input_2, 'k-', label="input_2")
plt.legend()
plt.show()

### 2. Normalize the Data

In [ ]:
# Get Range of data and Normalize the data
VALUES_RANGE = round(max(true_values)-min(true_values), 3)
VALUES_MEAN = round(np.mean(true_values), 3)
true_values = (true_values-VALUES_MEAN)/VALUES_RANGE

INPUT_1_RANGE = round(max(input_1)-min(input_1), 3)
INPUT_1_MEAN = round(np.mean(input_1), 3)
input_1 = (input_1-INPUT_1_MEAN)/INPUT_1_RANGE

INPUT_2_RANGE = round(max(input_2)-min(input_2), 3)
INPUT_2_MEAN = round(np.mean(input_2), 3)
input_2 = (input_2-INPUT_2_MEAN)/INPUT_2_RANGE

# calculate step size
STEP_SIZE = round(time[1]-time[0], 5)

# Print Values
print('VALUES_RANGE: ', VALUES_RANGE)
print('VALUES_MEAN: ', VALUES_MEAN)
print('INPUT_1_RANGE: ', INPUT_1_RANGE)
print('INPUT_1_MEAN: ', INPUT_1_MEAN)
print('INPUT_2_RANGE: ', INPUT_2_RANGE)
print('INPUT_2_MEAN: ', INPUT_2_MEAN)
print('STEP_SIZE: ', STEP_SIZE)

### 3. Split the Data and Create Dataset


In [ ]:
# 60% of our data for training and 20% for testing, remaining 20% for validation
train_split =  int(0.6 * len(time))
test_split = int(0.2*len(time) + train_split)

# Use np.split to chop data into three parts.
# True data
time_train, time_validate, time_test = np.split(time, [train_split,test_split])
true_values_train, true_values_validate, true_values_test = np.split(true_values, [train_split,test_split])
# input_1
input_1_train, input_1_validate, input_1_test = np.split(input_1, [train_split,test_split])
# input_2
input_2_train, input_2_validate, input_2_test = np.split(input_2, [train_split,test_split])

# Plot the data in each partition in different colors:
# True data
plt.figure(figsize=(15,3))
plt.title('True Values (logits)')
plt.plot(time_train, true_values_train, 'b-', label="Train")
plt.plot(time_test, true_values_test, 'r-', label="Test")
plt.plot(time_validate, true_values_validate, 'g-', label="Validate")
plt.legend()
plt.show()
# input_1
plt.figure(figsize=(15,3))
plt.title('Input 1')
plt.plot(time_train, input_1_train, 'b-', label="Train")
plt.plot(time_test, input_1_test, 'r-', label="Test")
plt.plot(time_validate, input_1_validate, 'g-', label="Validate")
plt.legend()
plt.show()
# input_2
plt.figure(figsize=(15,3))
plt.title('Input 2')
plt.plot(time_train, input_2_train, 'b-', label="Train")
plt.plot(time_test, input_2_test, 'r-', label="Test")
plt.plot(time_validate, input_2_validate, 'g-', label="Validate")
plt.legend()
plt.show()

## Model Design

### 1. Define the Model Hyperparameter

In [ ]:
# Model and Training Hyperparameters
UNITS = 32
ACTIVATION = 'relu'
EPOCHS = 1000
DROPOUT = 0.1
WINDOW_SIZE = 16
BATCH_SIZE_TRAIN = 64
BATCH_SIZE_INFERENCE = 1
NUM_FEATURES = 2
NUM_OUTPUTS = 1
LEARNING_RATE = 1e-6
LOSS = 'mean_squared_error'
METRIC = 'mae'
OPTIMIZER = tf.optimizers.Adam

### 2. Define Model

In [ ]:
def Model(batch_size, quantize_aware=False, weights=False):

    sequential_model = tf.keras.models.Sequential([
        
        # Input Layer 
        tf.keras.layers.InputLayer(input_shape=(WINDOW_SIZE, NUM_FEATURES), batch_size=batch_size),

        # RNN layer with quantization
        tf.keras.layers.LSTM(UNITS, activation=ACTIVATION, dropout=DROPOUT, return_sequences=True),
        tf.keras.layers.LSTM(UNITS, activation=ACTIVATION, dropout=DROPOUT),

        # Add Single Output Layer (perceptron)
        tf.keras.layers.Dense(NUM_OUTPUTS)])
        
    return sequential_model

# Function to compile Model
def Compile_Model(model):
    model.compile(loss=LOSS, optimizer=OPTIMIZER(learning_rate=LEARNING_RATE), metrics=[METRIC])

## Build Datasets

### 1. Create Function to build Dataset

In [ ]:
# Create widowed Dataset
def get_data(true_values, input_1, input_2, batch_size):

  # the length of data series (rounded with respect to batch size for Training)
  n_datapoints = (len(true_values)-WINDOW_SIZE)//batch_size*batch_size

  # set array with indices to loop through
  idx = np.arange(n_datapoints)

  # Get input and output Batch 
  input_1 = [input_1[i : i+WINDOW_SIZE] for i in idx]
  input_2 = [input_2[i : i+WINDOW_SIZE] for i in idx]
  output_data = [true_values[i+WINDOW_SIZE] for i in idx]

  # Interweaving of input data
  input_data = []
  for ii in range(n_datapoints):
    for jj in range(WINDOW_SIZE):
      input_data.append(input_1[ii][jj])
      input_data.append(input_2[ii][jj])

  # Reshape Data to wanted format
  input_data = np.reshape(input_data, [n_datapoints, WINDOW_SIZE, NUM_FEATURES])
  output_data = np.reshape(output_data, [n_datapoints, NUM_OUTPUTS])
      
      
  return input_data, output_data

### 2. Create Datasets for Training and Inference

In [ ]:
# Dataset for training
input_train, true_output_train = get_data(true_values_train, input_1_train, input_2_train, BATCH_SIZE_TRAIN)
input_val, true_output_val = get_data(true_values_validate, input_1_validate, input_2_validate, BATCH_SIZE_TRAIN)
input_test, true_output_test = get_data(true_values_test, input_1_test, input_2_test, BATCH_SIZE_TRAIN)

# Dataset for Inference
input_train_inf, true_output_train_inf = get_data(true_values_train, input_1_train, input_2_train, BATCH_SIZE_INFERENCE)
input_val_inf, true_output_val_inf = get_data(true_values_validate, input_1_validate, input_2_validate, BATCH_SIZE_INFERENCE)
input_test_inf, true_output_test_inf = get_data(true_values_test, input_1_test, input_2_test, BATCH_SIZE_INFERENCE)

## Train Base Model and Evaluate

### 1. Define Callbacks

In [ ]:
# Callback for to stop early
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=50)

# Callback to restore best Weight during training
early_stop_and_restore_weights = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=100, restore_best_weights=True)

### 2. Define Learining Rate Vector

In [ ]:
# init vector and order
learning_rates = [LEARNING_RATE]
order = -1

# append Learing rates from LEARNING_RATE to 10^2
while learning_rates[-1] < 10:
    order += 1
    for scale in range(2,12,2):
        learning_rates.append(learning_rates[0] * 10**order * scale)

### 4. Train with different Learning Rates

In [ ]:
# # init vector with calculated Mean Absolut Errors
# maes = []

# # Train Model for 100 different learining rates
# for learning_rate in learning_rates:

#   # set new learning rate
#   LEARNING_RATE = learning_rate

#   # build Model with defined Model Class
#   model_lr = Model(batch_size=BATCH_SIZE_TRAIN)

#   # Compile the model
#   Compile_Model(model_lr)

#   # Run training step through network
#   history = model_lr.fit( input_train, true_output_train,
#                           validation_data=(input_val, true_output_val),
#                           epochs=EPOCHS, 
#                           batch_size=BATCH_SIZE_TRAIN, 
#                           shuffle=True, 
#                           callbacks=[early_stopping],
#                           verbose=0)

#   # Get best mae on training for current learning rate
#   mae = min(history.history['val_mae'])*VALUES_RANGE
#   maes.append(mae)

#   # print MAE for this LR
#   print(f'learning rate: {learning_rate} -> MAE: {round(mae,3)} ~ {round(100*mae/VALUES_RANGE,3)}%')

#   # Stop learning if mae for last 3 LRs was nan
#   if math.isnan(mae):
#     break

### 5. Plot MAE over Learning Rates and select best one

In [ ]:
# # skip first and last Values to get better Plot
# skip_begin = 0
# skip_end = 3

# # Plot Error over Learing Rates
# plt.figure(figsize=(15,3))
# plt.semilogx(learning_rates[skip_begin:len(maes)-skip_end], maes[skip_begin:-skip_end], 'k')
# plt.title('Mean Absolut Error over Learningrate')
# plt.xlabel('Learning Rate')
# plt.ylabel('MAE')
# plt.show()

In [ ]:
# # select best learning rate
# lr_idx = [i for i, mae in enumerate(maes) if mae==min(maes)][0]
# LEARNING_RATE = learning_rates[lr_idx]
# print('BEST LEARNING RATE FOUND:', LEARNING_RATE)

LEARNING_RATE = 0.01

### 6. Learning with optimal Learining Rate

In [ ]:
# build Model with defined Model Class for Training
model = Model(batch_size=BATCH_SIZE_TRAIN)

# Compile the model
Compile_Model(model)

# Run training step through network
history = model.fit( input_train, true_output_train,
                        validation_data=(input_val, true_output_val),
                        epochs=EPOCHS, 
                        batch_size=BATCH_SIZE_TRAIN, 
                        shuffle=True,
                        callbacks=[early_stop_and_restore_weights],
                        verbose=2)

### 7. Fix Model Input Size for Inference and Show model summary

In [ ]:
# Build model again with batch size for inference
model_inference = Model(batch_size=BATCH_SIZE_INFERENCE)

# Set the trained weight for the new model
model_inference.set_weights(model.get_weights())

# Save model as Keras and tf model
model_inference.save(MODEL_KERAS, save_format="h5")
model_inference.save(MODEL_TF, save_format="tf")

# show Model summary
model_inference.summary()

### 8. Plot Metrics

#### 8.1 Plot Loss


In [ ]:
# Training Data
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# plot
SKIP = 3
plt.figure(figsize=(15,4))
plt.plot(np.arange(1, len(val_loss)+1, len(val_loss)/len(train_loss))[SKIP:], train_loss[SKIP:], 'k', label='Training loss')
plt.plot(np.arange(1, len(val_loss)+1)[SKIP:], val_loss[SKIP:], 'k--', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

#### 8.2 Predict output for all datasets and get Error

In [ ]:
# Make predictions based on all datasets
pred_values_train_tf = model_inference(input_train_inf)
pred_values_val_tf = model_inference(input_val_inf)
pred_values_test_tf = model_inference(input_test_inf)

# Calculate mae for all datasets
mae = tf.keras.metrics.mean_absolute_error
train_mae = mae(np.squeeze(pred_values_train_tf), np.squeeze(true_output_train_inf)).numpy()*VALUES_RANGE
val_mae = mae(np.squeeze(pred_values_val_tf), np.squeeze(true_output_val_inf)).numpy()*VALUES_RANGE
test_mae = mae(np.squeeze(pred_values_test_tf), np.squeeze(true_output_test_inf)).numpy()*VALUES_RANGE

# Print maes
print(f'TRAIN MAE: {round((train_mae),3)} ~ {round(100*train_mae/VALUES_RANGE,3)} %')
print(f'VALIDATION MAE: {round((val_mae),3)} ~ {round(100*val_mae/VALUES_RANGE,3)} %')
print(f'TEST MAE: {round((test_mae),3)} ~ {round(100*test_mae/VALUES_RANGE,3)} %')

#### 8.3 Plot predictions

In [ ]:
# Graph the predictions against the actual values
plt.figure(figsize=(15,4))

plt.title('Predictions vs Actual Values')

plt.plot(time_train, true_values_train, 'b', label='Train set')
plt.plot(time_train[WINDOW_SIZE:len(pred_values_train_tf)+WINDOW_SIZE], pred_values_train_tf, 'k', label='Predictions')
plt.plot(time_validate, true_values_validate, 'r', label='Validation Set')
plt.plot(time_validate[WINDOW_SIZE:len(pred_values_val_tf)+WINDOW_SIZE], pred_values_val_tf, 'k')
plt.plot(time_test, true_values_test, 'g', label='Test Set')
plt.plot(time_test[WINDOW_SIZE:len(pred_values_test_tf)+WINDOW_SIZE], pred_values_test_tf, 'k')

plt.legend()
plt.show()

## Generate and Train quantization aware Model

### 1. Define quantization config

In [ ]:
# RNN Quant Config
class Quant_Config(tfmot.quantization.keras.QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
      return [(layer.cell.recurrent_kernel, tfmot.quantization.keras.quantizers.LastValueQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]
    def get_activations_and_quantizers(self, layer):
      return [(layer.activation, tfmot.quantization.keras.quantizers.MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]
    def set_quantize_weights(self, layer, quantize_weights):
      layer.cell.recurrent_kernel = quantize_weights[0]
    def set_quantize_activations(self, layer, quantize_activations):
      layer.activations = quantize_activations[0]
    def get_output_quantizers(self, layer):
      return []
    def get_config(self):
      return {}

# LSTM layers should be quantized
def apply_quantization_to_LSTM(layer):
  if isinstance(layer, tf.keras.layers.LSTM):
    return tfmot.quantization.keras.quantize_annotate_layer(layer, Quant_Config())
  return layer

### 2. Train quantization aware model

In [ ]:
# anotate the lstm layers
quant_aware_model = tf.keras.models.clone_model(model,clone_function=apply_quantization_to_LSTM)

# Make model quant aware
quant_aware_model = tfmot.quantization.keras.quantize_annotate_model(quant_aware_model)
with tfmot.quantization.keras.quantize_scope({'Quant_Config': Quant_Config}):
    quant_aware_model = tfmot.quantization.keras.quantize_apply(quant_aware_model)

# quantization requires a recompile
Compile_Model(quant_aware_model)

# Train the quantization aware model
history_quant = quant_aware_model.fit( input_train, true_output_train,
                        validation_data=(input_val, true_output_val),
                        epochs=EPOCHS, 
                        batch_size=BATCH_SIZE_TRAIN, 
                        shuffle=True,
                        callbacks=[early_stop_and_restore_weights],
                        verbose=2)

### 3. Fix Model Input Size for Inference and show summary

In [ ]:
# anotate the lstm layers of the inference model
quant_aware_model_inference = tf.keras.models.clone_model(model_inference, clone_function=apply_quantization_to_LSTM)

# Make the inference model quant aware
quant_aware_model_inference = tfmot.quantization.keras.quantize_annotate_model(quant_aware_model_inference)
with tfmot.quantization.keras.quantize_scope({'Quant_Config': Quant_Config}):
    quant_aware_model_inference = tfmot.quantization.keras.quantize_apply(quant_aware_model_inference)

# Set the weights trained with quantization aware model
quant_aware_model_inference.set_weights(quant_aware_model.get_weights())

# Save model as Keras and tf model
quant_aware_model_inference.save(MODEL_KERAS_QUANT, save_format="h5")
quant_aware_model_inference.save(MODEL_TF_QUANT, save_format="tf")

# show Model summary
quant_aware_model_inference.summary()

### 4. Plot Metrics

#### 4.1 Plot Loss

In [ ]:
# Training Data
train_loss_quant = history_quant.history['loss']
val_loss_quant = history_quant.history['val_loss']

# plot
SKIP = 1
plt.figure(figsize=(15,4))
plt.plot(np.arange(1, len(val_loss_quant)+1, len(val_loss_quant)/len(train_loss_quant))[SKIP:], train_loss_quant[SKIP:], 'k', label='Training loss')
plt.plot(np.arange(1, len(val_loss_quant)+1)[SKIP:], val_loss_quant[SKIP:], 'k--', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

#### 4.2 Predict Output for all Datasets and get Error

In [ ]:
# Make predictions based on all datasets
pred_values_train_quant = quant_aware_model_inference(input_train_inf)
pred_values_val_quant = quant_aware_model_inference(input_val_inf)
pred_values_test_quant = quant_aware_model_inference(input_test_inf)

# Calculate mae for all datasets
train_mae_quant = mae(np.squeeze(pred_values_train_quant), np.squeeze(true_output_train_inf)).numpy()*VALUES_RANGE
val_mae_quant = mae(np.squeeze(pred_values_val_quant), np.squeeze(true_output_val_inf)).numpy()*VALUES_RANGE
test_mae_quant = mae(np.squeeze(pred_values_test_quant), np.squeeze(true_output_test_inf)).numpy()*VALUES_RANGE

# Print maes
print(f'TRAIN MAE: {round((train_mae_quant),3)} ~ {round(100*train_mae_quant/VALUES_RANGE,3)} %')
print(f'VALIDATION MAE: {round((val_mae_quant),3)} ~ {round(100*val_mae_quant/VALUES_RANGE,3)} %')
print(f'TEST MAE: {round((test_mae_quant),3)} ~ {round(100*test_mae_quant/VALUES_RANGE,3)} %')

#### 4.3 Plot Predictions

In [ ]:
# Graph the predictions against the actual values
plt.figure(figsize=(15,4))

plt.title('Predictions vs Actual Values')

plt.plot(time_train, true_values_train, 'b', label='Train set')
plt.plot(time_train[WINDOW_SIZE:len(pred_values_train_quant)+WINDOW_SIZE], pred_values_train_quant, 'k', label='Predictions Quantized Model')
plt.plot(time_validate, true_values_validate, 'r', label='Validation Set')
plt.plot(time_validate[WINDOW_SIZE:len(pred_values_val_quant)+WINDOW_SIZE], pred_values_val_quant, 'k')
plt.plot(time_test, true_values_test, 'g', label='Test Set')
plt.plot(time_test[WINDOW_SIZE:len(pred_values_test_quant)+WINDOW_SIZE], pred_values_test_quant, 'k')

plt.legend()
plt.show()


## Generate a TensorFlow Lite Model

### 1. Generate TF Lite Model

In [ ]:
# Init converter
converter_quant = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model_inference)
converter = tf.lite.TFLiteConverter.from_keras_model(model_inference)

# Set Optimizations
converter_quant.optimizations = [tf.lite.Optimize.DEFAULT]
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert and save the tf lite model
model_tflite_quant = converter_quant.convert()
model_tflite = converter.convert()
open(MODEL_TFLITE_QUANT, "wb").write(model_tflite_quant)
open(MODEL_TFLITE, "wb").write(model_tflite)

### 2. Compare Model Performance

**1. Define Functin for Inference**

In [ ]:
def predict_tflite(tflite_model, input):
  #  Load Test Data
  input_data = input.astype('float32')
  input_shape = (1, input_data.shape[1], input_data.shape[2])
  
  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.resize_tensor_input(0, input_shape);
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]["index"]
  output_details = interpreter.get_output_details()[0]["index"]
  
  # Invoke the interpreter
  predictions = []
  for input_datapoint in input_data:
    interpreter.set_tensor(input_details, [input_datapoint])
    interpreter.invoke()
    output = interpreter.tensor(output_details)
    predictions.append(output()[0])

  return predictions

**2. Predictions**

In [ ]:
# Calculate predictions
pred_values_test_tflite_quant = [predict_tflite(model_tflite_quant, np.expand_dims(input_sequence, axis=0)) for input_sequence in input_test_inf]
pred_values_test_tflite_quant = np.squeeze(pred_values_test_tflite_quant)
pred_values_test_tflite = [predict_tflite(model_tflite, np.expand_dims(input_sequence, axis=0)) for input_sequence in input_test_inf]
pred_values_test_tflite = np.squeeze(pred_values_test_tflite)

# Compare predictions
plt.clf()
plt.title('Comparison of various models against actual values')
plt.plot(time_test, true_values_test, 'k.', label='Actual values')
plt.plot(time_test[WINDOW_SIZE:len(pred_values_test_tf)+WINDOW_SIZE], pred_values_test_tf, 'bo', label='TF predictions')
plt.plot(time_test[WINDOW_SIZE:], pred_values_test_tflite_quant, 'r.', label='TFLite Quant predictions')
plt.plot(time_test[WINDOW_SIZE:], pred_values_test_tflite, 'g.', label='TFLite predictions')
plt.legend()
plt.show()

**2. Loss (MAE/Mean Absolut Error)**

In [ ]:
# Get mae of tflite model on train set
mae_tflite_quant = mae(np.squeeze(true_output_test_inf), pred_values_test_tflite_quant).numpy()*VALUES_RANGE
mae_tflite = mae(np.squeeze(true_output_test_inf), pred_values_test_tflite).numpy()*VALUES_RANGE

# Compare MAE
print('MAE BASE:', round((test_mae),3),'~',round(100*test_mae/VALUES_RANGE,3),'%')
print(f'MAE QUANT AWARE: {round((test_mae_quant),3)} ~ {round(100*test_mae_quant/VALUES_RANGE,3)} %')
print('MAE TF LITE:', round((mae_tflite),3),'~',round(100*mae_tflite/VALUES_RANGE,3),'%')
print('MAE TF LITE QUANT:', round((mae_tflite_quant),3),'~',round(100*mae_tflite_quant/VALUES_RANGE,3),'%')

## Generate a TensorFlow Lite for Microcontrollers Model
Convert the TensorFlow Lite quantized model into a C source file that can be loaded by TensorFlow Lite for Microcontrollers.

In [ ]:
# Convert Model to C
source_text_quant, _ = convert_bytes_to_c_source(model_tflite_quant, "model")
source_text, _ = convert_bytes_to_c_source(model_tflite, "model")

# Save C File as Header
with  open(MODEL_TFLITE_HEADER_QUANT,  'w')  as  file:
    file.write('#ifndef MODEL_DATA_H\n#define MODEL_DATA_H\n\n'+source_text_quant+'\n\n#endif //MODEL_DATA_H')

with  open(MODEL_TFLITE_HEADER,  'w')  as  file:
    file.write('#ifndef MODEL_DATA_H\n#define MODEL_DATA_H\n\n'+source_text+'\n\n#endif //MODEL_DATA_H')

    